In [25]:
'''
info_catch is to build up connection
'''

from bs4 import BeautifulSoup
import requests
import pandas as pd

def info_catch(url): 
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
               ' (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36',"accept": "*/*","accept-encoding": "gzip, deflate, br",
                "accept-language": "zh-CN,zh;q=0.9", "referer": "https://gongyi.qq.com/"}
    r = requests.get(url, headers = headers)
    html_contents = r.text
    html_soup = BeautifulSoup(html_contents, "html.parser")
    return html_soup

In [26]:
'''
trans_info is to crawl data
visual_info is to turn list into dataframe
write_info is to write dataframe to excel
'''

import pandas as pd
import json
import numpy as np
import random 

# xxx_l is a the list storing all the attributes we need to crawl
title_l = []
record_num_l = []
startTime_l = []
endTime_l = []
m_summary_l = []
proj_prov_l = []
cateName_l = []

fund_id_l = []
times_l = []
money_l = []
needMoney_l = []

fundName_l = []
eOrgName_l = []

#if we need to crawl fund or act status projects, these block need to be uncommented!!!
# def trans_info(result):
#     info = str(result).split('"id"') #treat the json file as a string. Split the string by id(The beginning of each project)
#     for i in range(1,len(info)): #complement all the sub strings to make it as a json format
#         if i != 10: #Not the last string
#             info[i] = '{"id"'+info[i][:-2] #act & fund
#         if i == 10: #The last string 
#             info[i] = '{"id"'+info[i][:-3] #act & fund
#         try: #handle the json format exception
#             info_dict = json.loads(info[i])
#             title_l.append(info_dict["title"])
#             eOrgName_l.append(info_dict["eOrgName"])
#             endTime_l.append(info_dict["endTime"])
#             m_summary_l.append(info_dict["m_summary"])
#             needMoney_l.append(float(info_dict['needMoney'])/100)
#             fundName_l.append(info_dict['fundName'])
#             record_num_l.append(info_dict['record_num'])
#             money_l.append(float(info_dict['stat']['money'])/100)
#             times_l.append(info_dict['stat']['times'])
#             startTime_l.append(info_dict['startTime'])
#             fund_id_l.append(info_dict['fund_id'])
#             proj_prov_l.append(info_dict['proj_province'])
#             cateName_l.append(info_dict['cateName'])
#         except Exception as e:
#             print(e)
#     return 

#if we need to crawl end status projects, these block need to be uncommented!!!
def trans_info(result):    
    info = str(result).split('"id"') #treat the json file as a string. Split the string by id(The beginning of each project)
    for i in range(1,len(info)): #complement all the sub strings to make it as a json format
        a = info[i][-13:]
        if i != 10:  #Not the last string
            if info[i][-13:] == a:
                info[i] = '{"id"'+info[i][:-13]
            else:  
                info[i] = '{"id"'+info[i][:-2]
        if i == 10: #The last string
            info[i] = '{"id"'+info[i][:-3]
        try: #handle the json format exception
            info_dict = json.loads(info[i])
            title_l.append(info_dict["title"])
            eOrgName_l.append(info_dict["eOrgName"])
            endTime_l.append(info_dict["endTime"])
            m_summary_l.append(info_dict["m_summary"])
            needMoney_l.append(float(info_dict['needMoney'])/100)
            fundName_l.append(info_dict['fundName'])
            record_num_l.append(info_dict.get('record_num',None))
            money_l.append(float(info_dict['stat']['money'])/100)
            times_l.append(info_dict['stat']['times'])
            startTime_l.append(info_dict['startTime'])
            fund_id_l.append(info_dict['fund_id'])
            cateName_l.append(info_dict['cateName'])
            proj_prov_l.append(info_dict['proj_province'])
        except Exception as e:
            print(e)
    return 

def visual_info(a,b,c,d,e,f,g,h,i,j,k,l,m): #store as a dataframe
    infoDF = pd.DataFrame({'Project_Name':a,'Project_id':b,
                           'Start_date':c,'End_date':d,
                           'Introduction':e,'Project_province':f,
                           'Category':g,'Fund_id':h,
                           'Donor_num':i,'Cum_amount':j,
                           'Target_amount':k,'Reci_name':l,'Ini_name':m,})
    return infoDF

def write_info(df_data): #write the dataframe to excel
    #writer = pd.ExcelWriter('fund_preprocess.xlsx')#fund
    #writer = pd.ExcelWriter('act.xlsx') #act
    writer = pd.ExcelWriter('end.xlsx') #end
    df_data.to_excel(writer,'Sheet1',encoding='gb2312')
    writer.save()

In [27]:
'''
It is the main function
build up connection - crawl - dataframe - excel
'''
import time, random
#fund
#orig_url = "https://ssl.gongyi.qq.com/cgi-bin/WXSearchCGI?ptype=stat&s_status=1&jsoncallback=_CallbackSearch&s_status=1&s_tid=&s_puin=&s_fid=&s_key=&p="
#act
#orig_url = "https://ssl.gongyi.qq.com/cgi-bin/WXSearchCGI?ptype=stat&s_status=2&jsoncallback=_CallbackSearch&s_status=2&s_tid=&s_puin=&s_fid=&s_key=&p="
#end
orig_url = "https://ssl.gongyi.qq.com/cgi-bin/WXSearchCGI?ptype=stat&s_status=3&jsoncallback=_CallbackSearch&s_status=3&s_tid=&s_puin=&s_fid=&s_key=&p="

for i in range(1,105): #total no more than 105 pages 
    url = orig_url+str(i)+"&_="+str(time.time()).replace(".","")[:13] #turn pages
    print(i,url)
    time.sleep(random.uniform(3, 7)) #avoid banning
    result = info_catch(url) #build up connection
    trans_info(result) #crawl data
    #translate to dataframe
    a = visual_info(title_l,record_num_l,startTime_l,endTime_l,m_summary_l,proj_prov_l,cateName_l,fund_id_l,times_l,money_l,needMoney_l,eOrgName_l,fundName_l)

display(a)
write_info(a) #write to excel

1 https://ssl.gongyi.qq.com/cgi-bin/WXSearchCGI?ptype=stat&s_status=3&jsoncallback=_CallbackSearch&s_status=3&s_tid=&s_puin=&s_fid=&s_key=&p=1&_=1621759259025
2 https://ssl.gongyi.qq.com/cgi-bin/WXSearchCGI?ptype=stat&s_status=3&jsoncallback=_CallbackSearch&s_status=3&s_tid=&s_puin=&s_fid=&s_key=&p=2&_=1621759262886
Unterminated string starting at: line 1 column 4234 (char 4233)
Expecting ',' delimiter: line 1 column 2808 (char 2807)
3 https://ssl.gongyi.qq.com/cgi-bin/WXSearchCGI?ptype=stat&s_status=3&jsoncallback=_CallbackSearch&s_status=3&s_tid=&s_puin=&s_fid=&s_key=&p=3&_=1621759266609
Expecting ',' delimiter: line 1 column 3498 (char 3497)
Expecting ',' delimiter: line 1 column 3777 (char 3776)
4 https://ssl.gongyi.qq.com/cgi-bin/WXSearchCGI?ptype=stat&s_status=3&jsoncallback=_CallbackSearch&s_status=3&s_tid=&s_puin=&s_fid=&s_key=&p=4&_=1621759273274
5 https://ssl.gongyi.qq.com/cgi-bin/WXSearchCGI?ptype=stat&s_status=3&jsoncallback=_CallbackSearch&s_status=3&s_tid=&s_puin=&s_fid=&

Unterminated string starting at: line 1 column 4833 (char 4832)
48 https://ssl.gongyi.qq.com/cgi-bin/WXSearchCGI?ptype=stat&s_status=3&jsoncallback=_CallbackSearch&s_status=3&s_tid=&s_puin=&s_fid=&s_key=&p=48&_=1621759516432
49 https://ssl.gongyi.qq.com/cgi-bin/WXSearchCGI?ptype=stat&s_status=3&jsoncallback=_CallbackSearch&s_status=3&s_tid=&s_puin=&s_fid=&s_key=&p=49&_=1621759520773
Expecting ',' delimiter: line 1 column 3161 (char 3160)
Expecting ',' delimiter: line 1 column 2831 (char 2830)
50 https://ssl.gongyi.qq.com/cgi-bin/WXSearchCGI?ptype=stat&s_status=3&jsoncallback=_CallbackSearch&s_status=3&s_tid=&s_puin=&s_fid=&s_key=&p=50&_=1621759527263
51 https://ssl.gongyi.qq.com/cgi-bin/WXSearchCGI?ptype=stat&s_status=3&jsoncallback=_CallbackSearch&s_status=3&s_tid=&s_puin=&s_fid=&s_key=&p=51&_=1621759534071
52 https://ssl.gongyi.qq.com/cgi-bin/WXSearchCGI?ptype=stat&s_status=3&jsoncallback=_CallbackSearch&s_status=3&s_tid=&s_puin=&s_fid=&s_key=&p=52&_=1621759541315
53 https://ssl.gong

96 https://ssl.gongyi.qq.com/cgi-bin/WXSearchCGI?ptype=stat&s_status=3&jsoncallback=_CallbackSearch&s_status=3&s_tid=&s_puin=&s_fid=&s_key=&p=96&_=1621759777644
97 https://ssl.gongyi.qq.com/cgi-bin/WXSearchCGI?ptype=stat&s_status=3&jsoncallback=_CallbackSearch&s_status=3&s_tid=&s_puin=&s_fid=&s_key=&p=97&_=1621759783524
98 https://ssl.gongyi.qq.com/cgi-bin/WXSearchCGI?ptype=stat&s_status=3&jsoncallback=_CallbackSearch&s_status=3&s_tid=&s_puin=&s_fid=&s_key=&p=98&_=1621759788263
99 https://ssl.gongyi.qq.com/cgi-bin/WXSearchCGI?ptype=stat&s_status=3&jsoncallback=_CallbackSearch&s_status=3&s_tid=&s_puin=&s_fid=&s_key=&p=99&_=1621759792022
100 https://ssl.gongyi.qq.com/cgi-bin/WXSearchCGI?ptype=stat&s_status=3&jsoncallback=_CallbackSearch&s_status=3&s_tid=&s_puin=&s_fid=&s_key=&p=100&_=1621759798527
101 https://ssl.gongyi.qq.com/cgi-bin/WXSearchCGI?ptype=stat&s_status=3&jsoncallback=_CallbackSearch&s_status=3&s_tid=&s_puin=&s_fid=&s_key=&p=101&_=1621759805243
102 https://ssl.gongyi.qq.com/

,Project_Name,Project_id,Start_date,End_date,Introduction,Project_province,Category,Fund_id,Donor_num,Cum_amount,Target_amount,Reci_name,Ini_name
0,赡养父母养育妻儿,53100000500019795AA18044,2018-01-10,2018-04-10,身患白血病无力支持昂贵的医疗费用。被迫出院,安徽省,疾病救助,46,157,5266.12,400000.0,张胜玲,中国社会福利基金会
1,帮两岁小白战病魔,53100000500019795AA18091,2018-01-08,2018-04-08,两岁的吴雨诺急需医药费盼救治,内蒙古自治区,疾病救助,46,103,2112.12,400000.0,吴磊磊,中国社会福利基金会
2,地贫男孩不放弃路,53100000500019795AA17145,2017-12-29,2018-03-29,重型B-地中海贫血男孩，一直不放弃不妥协,广西壮族自治区,疾病救助,46,90,2623.43,100000.0,李国华,中国社会福利基金会
3,病债未清，又遇人祸,53100000500019795AA17145,2017-12-29,2018-03-29,还未还清父母治疗癌症外债，孩子又被五保老人碰撞,安徽省,疾病救助,46,146,2131.00,300000.0,闻庆军,中国社会福利基金会
4,铁箍女压不垮的脊梁,5332000050917103XBA18040,2018-03-16,2018-03-26,这位“钢钉刺骨”、一直与生命赛跑的妈妈，人见人心疼！,四川省,疾病救助,40,545,15035.87,100000.0,万学强,爱德基金会
...,...,...,...,...,...,...,...,...,...,...,...,...,...
973,血癌女孩为生命呐喊,None,2017-06-23,2017-08-31,21岁的蒋月莹白血病化疗两年负债累累，目前复发急需移植,河北省,疾病救助,101,111,1311.91,300000.0,深圳市德义爱心促进会,深圳市慈善会
974,老兵“小钢炮”,None,2017-06-24,2017-08-31,老兵生活困难，需要医药救助。,河北省,扶贫救灾,133,168,2740.45,15000.0,于殿鹏,河北慈善联合基金会
975,81岁孤寡老人需帮助,None,2017-06-23,2017-08-31,老人现在孤苦伶仃的一个人生活,河北省,扶贫救灾,133,96,890.72,12000.0,于殿鹏,河北慈善联合基金会
976,儿童手足伤残救助,None,2017-06-26,2017-08-31,为贫困家庭手足伤残患儿提供手术经费资助。,重庆市,疾病救助,191,32,154.05,100000.0,重庆儿童救助基金会,重庆儿童救助基金会


In [34]:
'''
drop duplicate initiator and recipient
'''

import pandas as pd

#gather all the initiator and recipient in three excel files into two dataframes
df1 = pd.read_excel(r'fund_preprocess.xlsx')
Reci_name_f = df1['Reci_name']
Ini_name_f = df1['Ini_name']

df2 = pd.read_excel(r'act.xlsx')
Reci_name_a = df2['Reci_name']
Ini_name_a = df2['Ini_name']

df3 = pd.read_excel(r'end.xlsx')
Reci_name_e = df3['Reci_name']
Ini_name_e = df3['Ini_name']

#Reci_total = pd.concat([Reci_name_f,Reci_name_a,Reci_name_e],axis = 0)
Ini_total = pd.concat([Ini_name_f,Ini_name_a,Ini_name_e],axis = 0)

#drop duplicate value of two dataframe
#Reci_total = Reci_total.drop_duplicates()
Ini_total = Ini_total.drop_duplicates()

#Reci_contact_l = []
Ini_contact_l = []

'''
add phone number of each initiator and recipient by randomly create 11 digits phone number starting with number 1
and append to the lists as well
'''
# for i in range(Reci_total.shape[0]):
#     Reci_contact_l.append(str(1)+str(random.random()*10000000000).replace(".","")[:10])
#     i+=1               
for i in range(Ini_total.shape[0]):
    Ini_contact_l.append(str(1)+str(random.random()*10000000000).replace(".","")[:10])
    i+=1
'''
turn the lists into dataframes
'''
#Reci = pd.DataFrame({'Reci_name':Reci_total,'Reci_contact':Reci_contact_l})
Ini = pd.DataFrame({'Ini_name':Ini_total,'Ini_contact':Ini_contact_l})

'''
write the dataframes to excel
'''
# writer = pd.ExcelWriter('Recipient.xlsx')#Recipient
# Reci.to_excel(writer,'Sheet1',encoding='gb2312')
# writer.save()

writer = pd.ExcelWriter('Initiator.xlsx')#Initiator
Ini.to_excel(writer,'Sheet1',encoding='gb2312')
writer.save()

In [38]:
'''
Randomly create unique Project_id
'''
df = pd.read_excel(r'Project.xlsx')
import string

new_proj_id = []

#randomly create unique Project_id in format
for i in range(df.shape[0]):
    s1 = string.ascii_uppercase
    s2 = string.ascii_uppercase
    new_proj_id.append(str(53)+str(random.random()*100000000000000).replace(".","")[:15]+str(random.choice(s1))+str(random.choice(s2))+str(random.random()*10000).replace(".","")[:5])
    
df['Project_id'] = new_proj_id

#update excel
writer = pd.ExcelWriter('Project.xlsx')
df.to_excel(writer,"Sheet1",encoding='gb2312')
writer.save()

In [3]:
'''
Randomly generate User infomation 
'''
import string
import random
import pandas as pd

User_name_l = []
User_id_l = []
Password_l = []
Role_l = [] 

#to generate 50010 users with name and unique id
for i in range(0,50010):
    s1 = string.ascii_lowercase
    s2 = string.ascii_uppercase
    num = str(random.random()*100000).replace(".","")
    User_name_l.append(str(random.choice(s2))+str(random.choice(s1))+str(random.choice(s1))+num[4:7])
    if i<50008:
        User_id_l.append('UD'+str(i))
        Role_l.append('Donor')
    else:
        User_id_l.append('UA'+str(i-50008))
        Role_l.append('Admin')
    Password_l.append(num[:1]+str(random.choice(s1))+num[1:5]+str(random.choice(s2))+num[5:8])

#turn list in dataframe
info = pd.DataFrame({'User_name':User_name_l,'User_id':User_id_l,'Password':Password_l,'Role':Role_l})

#write to excel
writer = pd.ExcelWriter('User.xlsx')
info.to_excel(writer,"Sheet1",encoding='gb2312')
writer.save()

In [ ]:
'''
randomly generate fund amount for existing donors
(Because we can not crawl this infomation)
'''
import pandas as pd
import random

fund = pd.read_excel(r'Fund.xlsx')
donor = pd.read_excel(r'Donor.xlsx')
Donor_id_l = donor['Donor_id']
proj_id_orig = fund['Project_id']

Amount_l = []
Project_id_l = []

#randomly generate fund aomunt no larger than 100.00
for i in range(len(Donor_id_l)):
    Amount_l.append(str(random.random()*10000).replace(".","")[:2]+".00")
    Project_id_l.append(fund['Project_id'][i%len(proj_id_orig)])

#turn list into dataframe
df = pd.DataFrame({'Donor_id':Donor_id_l,'Amount':Amount_l,'Project_id':Project_id_l})
#write to excel
writer = pd.ExcelWriter('Donate.xlsx')
df.to_excel(writer,"Sheet1",encoding='gb2312')
writer.save()